In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Flatten, Dense, Input, concatenate
from tensorflow.keras.optimizers import SGD, Adam
import matplotlib .pyplot as plt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import accuracy_score

import json
import cv2
import zipfile
import shutil
import os
import glob
import tarfile

## 트레이닝 파일 압축 해제

In [ ]:
file_name = ["c착과군", "e열매", "d1꽃_개화군", "a1생장길이", "b개화군", "d2꽃_착과군", "f만개꽃", "a3줄기두께",
            "a4엽장엽폭", "a2화방높이"]
file = []

file_path="C:/Users/acorn/Downloads/097.지능형 스마트팜 통합 데이터(토마토)/01.데이터/1.Training/라벨링데이터/"

## 압축파일 리스트에 담기

for i in file_name:
    file.append("TL_{}.zip".format(i))
    
### 리스트를 for문 돌려서 압축파일 풀기
for f in file:
    with zipfile.ZipFile(file_path+f) as target_file:
        target_file.extractall(file_path)

## 밸리데이션 파일 압축 해제

In [ ]:
file_name = ["c착과군", "e열매", "d1꽃_개화군", "a1생장길이", "b개화군", "d2꽃_착과군", "f만개꽃", "a3줄기두께",
            "a4엽장엽폭", "a2화방높이"]
file2 = []
file_path2="C:/Users/acorn/Downloads/097.지능형 스마트팜 통합 데이터(토마토)/01.데이터/2.Validation/라벨링데이터/"
## 압축파일 리스트에 담기

for i in file_name:
    file2.append("VL_{}.zip".format(i))

### 리스트를 for문 돌려서 압축파일 풀기
for f in file2:
    with zipfile.ZipFile(file_path2+f) as target_file:
        target_file.extractall(file_path2)

## 트레이닝 JSON 파일 변수에 저장

In [ ]:
raw_json = []

# 리스트에 폴더안에 파일 이름 담기
for i in range(len(file_name)):
    raw_json.append(glob.glob(file_path+"{}/*.json".format(file_name[i])))
raw_json

In [ ]:
raw_f = []

# 리스트를 for문 돌려서 JSON파일 읽어와 리스트에 담기
for i in range(len(raw_json)):
    for k in range(len(raw_json[i])):
        with open(raw_json[i][k], "r", encoding="utf-8") as j :
            raw_f.append(json.loads(j.read()))

In [ ]:
len(raw_f)

## 밸리데이션 JSON 파일 변수에 저장

In [ ]:
raw_json2 = []
# 리스트에 폴더안에 파일 이름 담기

for i in range(len(file_name)):
    raw_json2.append(glob.glob(file_path2+"{}/*.json".format(file_name[i])))
raw_json2

In [ ]:
raw_f2 = []
# 리스트를 for문 돌려서 JSON파일 읽어와 리스트에 담기

for i in range(len(raw_json2)):
    for k in range(len(raw_json2[i])):
        with open(raw_json2[i][k], "r", encoding="utf-8") as j2 :
            raw_f2.append(json.loads(j2.read()))

In [ ]:
len(raw_f2)

## JSON파일 DataFrame으로 변환

In [ ]:
def mk_tomato_df(raw_f):
    tomato_list = []
    for i in tqdm(range(len(raw_f))):
        data = {
        'imagePath': raw_f[i]['imagePath'],
        'imageData': raw_f[i]['imageData'],
        'imageHeight': raw_f[i]['imageHeight'],
        'imageWidth': raw_f[i]['imageWidth'],
        'fileVersion': raw_f[i]['file_attributes']['fileVersion'],
        'farmId': raw_f[i]['file_attributes']['farmId'],
        'week': raw_f[i]['file_attributes']['week'],
        'number': raw_f[i]['file_attributes']['number'],
        'type': raw_f[i]['file_attributes']['type'],
        'flowerCluster': raw_f[i]['file_attributes']['flowerCluster'],
        'date': raw_f[i]['file_attributes']['date']
        }
        tomato_list.append(data)
        df = pd.DataFrame(tomato_list)
    return df
    
    
## df head 확인
mk_tomato_df(raw_f).head()

## 이미지 파일 압축 해제

In [ ]:
file_name = ["c착과군", "e열매", "d1꽃_개화군", "a1생장길이", "b개화군", "d2꽃_착과군", "f만개꽃", "a3줄기두께",
            "a4엽장엽폭", "a2화방높이"]

file_path = "D:/097.지능형 스마트팜 통합 데이터(토마토)/01.데이터/1.Training/원천데이터/"

file= []
for name in file_name:
    file.append("TS_{}.zip".format(name))
for f in tqdm(file):
    with zipfile.ZipFile(file_path+f) as target_file:
        target_file.extractall(file_path)

## 토마토 CSV파일을 DF파일로 저장하기(tom1, 4, 5, 6)

In [ ]:
name = ["00_(환경제어기, 양액기)", "01_(함수율저울,열화상센서,작물환경센서)"]
file_path = "D:/097.지능형 스마트팜 통합 데이터(토마토)/01.데이터/1.Training/원천데이터/TS_Timeseries"
file_list = []
env_list = []
dfs = []
to_file = os.path.join(file_path, "tom5")

# 경로 지정해서 리스트에 담기
for env in name:
    to_file_env = os.path.join(to_file, env)
    env_list.append(to_file_env)

# 파일들 이름 리스트에 저장하기
for file in env_list:
    file_name = glob.glob(file+"/*csv")
    
    for names in file_name:
        file_list.append(names)

## CSV파일 읽어올때 인코딩 오류가 있어서 해결하고 불러온 다음 DF에 담기
for i in range(len(file_list)):
    
    try:
        df = pd.read_csv(file_list[i], encoding="cp949")
        
    except UnicodeDecodeError as e :
        
        df = pd.read_csv(file_list[i], encoding="utf-8")
        
    dfs.append(df)

## 전처리

In [ ]:
## 컬럼 순서대로 맞추고 평균값 컬럼 생성하기

df_mean = (dfs[4]["내부PT100온도센서1번건구"]+dfs[4]["내부PT100온도센서2번습구"]+dfs[4]["내부PT100온도센서3번"])/3
df_mean["일일 평균온도(℃)"] = df_mean

dfs[0].rename(columns = {" 누적일사(J) " : "누적일사량(J/㎠/day)"}, inplace=True)

df_1 = pd.concat([dfs[0]["누적일사량(J/㎠/day)"], df_mean["일일 평균온도(℃)"], dfs[4]["내부이산화탄소"], dfs[4]["EC급액"], dfs[4]["pH급액"]], axis=1)